In [ ]:
# Poisson problem on a unit square with Dirichlet boundary conditions
# Manufactured solution.

import numpy as np
import eztfem as ezt
from func import func
from scipy.sparse.linalg import spsolve

In [ ]:
# create mesh
mesh = ezt.quadrilateral2d([20, 20], 'quad9')

In [ ]:
# define the problem
elementdof = np.array([[2, 2, 2, 2, 2, 2, 2, 2, 2],
                       [1, 0, 1, 0, 1, 0, 1, 0, 0],
                       [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=2)

In [ ]:
# user struct for setting problem coefficients, ...
user = ezt.User()
user.coorsys = 0
user.mu = 1
user.funcnr = 0
user.func = func  # not used when funcnr == 0

In [ ]:
# define Gauss integration and basis functions
shape = 'quad'
user.xr, user.wg = ezt.gauss_legendre(shape, n=3)
user.phi, user.dphi = ezt.basis_function(shape, 'Q2', user.xr)
user.psi, _ = ezt.basis_function(shape, 'Q1', user.xr)

In [ ]:
# assemble the system matrix and vector
A, f = ezt.build_system(mesh, problem, ezt.stokes_elem, user)

In [ ]:
# define essential boundary conditions (Dirichlet)
iess = ezt.define_essential(mesh, problem, 'curves', [0, 1, 2, 3], degfd=0)
iess = ezt.define_essential(mesh, problem, 'curves', [0, 1, 2, 3], degfd=1,
                            iessp=iess)
iess = ezt.define_essential(mesh, problem, 'points', [0], physq=1,
                            iessp=iess)


In [ ]:
# fill values for the essential boundary conditions
uess = ezt.fill_system_vector(mesh, problem, 'curves', [2], func, funcnr=5)

# apply essential boundary conditions to the system
ezt.apply_essential(A, f, uess, iess)

In [ ]:
# solve the system
u = spsolve(A.tocsr(), f)

In [ ]:
# Pressure in all nodes for plotting
xr = ezt.refcoor_nodal_points(mesh)
user.psi, _ = ezt.basis_function('quad', 'Q1', xr)
user.u = u
pressure = ezt.deriv_vector(mesh, problem, ezt.stokes_pressure, user)

In [ ]:
# derivatives of the velocity
xr = ezt.refcoor_nodal_points(mesh)
user.phi, user.dphi = ezt.basis_function('quad', 'Q2', xr)
user.u = u

user.comp = 0  # dudx
dudx = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 1  # dudy
dudy = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 2  # dvdx
dvdx = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 3  # dvdy
dvdy = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 4  # dvdx - dudy = vorticity
omega = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 6  # divu, divergence of the velocity field
divu = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)
user.comp = 7  # gammadot, effective strain rate = sqrt(2II_D)
gammadot = ezt.deriv_vector(mesh, problem, ezt.stokes_deriv, user)

print(max(omega.u))

In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:
# plot the mesh in Pyvista
ezt.plot_mesh_pv(mesh_pv, show_edges=True, window_size=(800, 400))

In [ ]:
# plot the x-velocity field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400), physq=0, degfd=0)

In [ ]:
# plot contours of the x-velocity
ezt.plot_sol_contour(mesh_pv, problem, u, physq=0, degfd=0, nlevels=20)

In [ ]:
# quiver plot of the velocity field
ezt.plot_quiver(mesh_pv, problem, u, window_size=(800, 400), physq=0, 
                scale=0.1)

In [ ]:
# plot the effective strain rate vector
ezt.plot_vector(mesh_pv, problem, gammadot)

In [ ]:
# contour plot of the effective strain rate
ezt.plot_vector_contours(mesh_pv, problem, gammadot, nlevels=8)

In [ ]:
# sample effective strain rate along a line
points = [[0.0, 0.3, 0.0], [1.0, 0.3, 0.0]]
_ = ezt.plot_vector_over_line(mesh_pv, problem, gammadot, points,
                              plot_mesh=False)